# 特征缩放

本笔记本实现了特征缩放的两种方法：标准化和归一化，并展示了特征缩放对梯度下降性能的影响。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. 生成不同尺度的特征数据

In [ ]:
def generate_data_with_different_scales():
    """
    生成具有不同尺度特征的模拟数据
    
    返回:
    X: 特征数据 (n_samples, 2)，两个特征尺度差异很大
    y: 目标值 (n_samples, 1)
    """
    n_samples = 100
    # 生成大范围特征 (0-100)
    X1 = np.random.rand(n_samples, 1) * 100  
    # 生成小范围特征 (0-1)
    X2 = np.random.rand(n_samples, 1) * 1    
    # 合并特征
    X = np.hstack((X1, X2))
    # 生成目标值
    y = 2 * X1 + 3 * X2 + np.random.randn(n_samples, 1) * 5
    return X, y

# 生成数据
X, y = generate_data_with_different_scales()
print(f"数据形状: X={X.shape}, y={y.shape}")
print(f"特征1范围: {np.min(X[:, 0]):.2f} 到 {np.max(X[:, 0]):.2f}")
print(f"特征2范围: {np.min(X[:, 1]):.2f} 到 {np.max(X[:, 1]):.2f}")

## 2. 可视化原始特征分布

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(X[:, 0], bins=20, alpha=0.6)
plt.xlabel('特征1值')
plt.ylabel('频率')
plt.title('特征1分布 (大范围)')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.hist(X[:, 1], bins=20, alpha=0.6)
plt.xlabel('特征2值')
plt.ylabel('频率')
plt.title('特征2分布 (小范围)')
plt.grid(True)

plt.tight_layout()
plt.show()

## 3. 标准化 (Standardization)

In [ ]:
def normalize_features(X):
    """
    对特征进行标准化处理
    
    参数:
    X: 特征矩阵 (m, n)
    
    返回:
    X_normalized: 标准化后的特征矩阵
    mean: 每个特征的均值
    std: 每个特征的标准差
    """
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X_normalized = (X - mean) / std
    return X_normalized, mean, std

# 标准化特征
X_normalized, mean, std = normalize_features(X)
print(f"标准化后特征1范围: {np.min(X_normalized[:, 0]):.2f} 到 {np.max(X_normalized[:, 0]):.2f}")
print(f"标准化后特征2范围: {np.min(X_normalized[:, 1]):.2f} 到 {np.max(X_normalized[:, 1]):.2f}")
print(f"特征均值: {mean}")
print(f"特征标准差: {std}")

## 4. 归一化 (Min-Max Scaling)

In [ ]:
def min_max_scaling(X):
    """
    对特征进行归一化处理
    
    参数:
    X: 特征矩阵 (m, n)
    
    返回:
    X_scaled: 归一化后的特征矩阵
    min_val: 每个特征的最小值
    max_val: 每个特征的最大值
    """
    min_val = np.min(X, axis=0)
    max_val = np.max(X, axis=0)
    X_scaled = (X - min_val) / (max_val - min_val)
    return X_scaled, min_val, max_val

# 归一化特征
X_minmax, min_val, max_val = min_max_scaling(X)
print(f"归一化后特征1范围: {np.min(X_minmax[:, 0]):.2f} 到 {np.max(X_minmax[:, 0]):.2f}")
print(f"归一化后特征2范围: {np.min(X_minmax[:, 1]):.2f} 到 {np.max(X_minmax[:, 1]):.2f}")
print(f"特征最小值: {min_val}")
print(f"特征最大值: {max_val}")

## 5. 可视化缩放后的特征分布

In [ ]:
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.scatter(X[:, 0], X[:, 1], alpha=0.6)
plt.xlabel('特征1')
plt.ylabel('特征2')
plt.title('原始特征分布')
plt.grid(True)

plt.subplot(1, 3, 2)
plt.scatter(X_normalized[:, 0], X_normalized[:, 1], alpha=0.6)
plt.xlabel('标准化特征1')
plt.ylabel('标准化特征2')
plt.title('标准化后特征分布')
plt.grid(True)

plt.subplot(1, 3, 3)
plt.scatter(X_minmax[:, 0], X_minmax[:, 1], alpha=0.6)
plt.xlabel('归一化特征1')
plt.ylabel('归一化特征2')
plt.title('归一化后特征分布')
plt.grid(True)

plt.tight_layout()
plt.show()

## 6. 梯度下降实现

In [ ]:
def gradient_descent(X, y, theta, learning_rate, n_iterations):
    """
    使用梯度下降算法训练线性回归模型
    
    参数:
    X: 特征矩阵 (m, n+1)，包含偏置项
    y: 目标值向量 (m, 1)
    theta: 初始参数向量 (n+1, 1)
    learning_rate: 学习率
    n_iterations: 迭代次数
    
    返回:
    theta: 学习后的参数向量
    cost_history: 每次迭代的代价函数值
    """
    m = len(y)  # 样本数量
    cost_history = np.zeros(n_iterations)  # 记录代价函数历史
    
    for i in range(n_iterations):
        # 计算预测值
        predictions = X.dot(theta)
        # 计算误差
        errors = predictions - y
        # 计算梯度
        gradients = (1/m) * X.T.dot(errors)
        # 更新参数
        theta = theta - learning_rate * gradients
        # 记录当前代价函数值
        cost_history[i] = (1/(2*m)) * np.sum(np.square(errors))
    
    return theta, cost_history

## 7. 比较不同特征缩放方法对梯度下降的影响

In [ ]:
# 为不同缩放方法的特征添加偏置项
X_b_original = np.c_[np.ones((len(X), 1)), X]
X_b_normalized = np.c_[np.ones((len(X_normalized), 1)), X_normalized]
X_b_minmax = np.c_[np.ones((len(X_minmax), 1)), X_minmax]

# 初始化参数
theta_initial = np.random.randn(3, 1)
n_iterations = 1000

# 训练模型
# 注意：原始特征需要更小的学习率
theta_original, cost_history_original = gradient_descent(X_b_original, y, theta_initial, 0.000001, n_iterations)
theta_normalized, cost_history_normalized = gradient_descent(X_b_normalized, y, theta_initial, 0.1, n_iterations)
theta_minmax, cost_history_minmax = gradient_descent(X_b_minmax, y, theta_initial, 0.1, n_iterations)

## 8. 可视化训练过程对比

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(n_iterations), cost_history_original, label='原始特征')
plt.plot(range(n_iterations), cost_history_normalized, label='标准化特征')
plt.plot(range(n_iterations), cost_history_minmax, label='归一化特征')
plt.xlabel('迭代次数')
plt.ylabel('代价函数值')
plt.title('特征缩放对梯度下降的影响')
plt.legend()
plt.grid(True)
plt.show()

## 9. 模型性能评估

In [ ]:
print("梯度下降结果:")
print(f"原始特征最终代价: {cost_history_original[-1]:.4f}")
print(f"标准化特征最终代价: {cost_history_normalized[-1]:.4f}")
print(f"归一化特征最终代价: {cost_history_minmax[-1]:.4f}")

print("\n学习到的参数:")
print(f"原始特征: {theta_original.ravel()}")
print(f"标准化特征: {theta_normalized.ravel()}")
print(f"归一化特征: {theta_minmax.ravel()}")

## 10. 特征缩放总结

In [ ]:
print("特征缩放的优点:")
print("1. 加速梯度下降的收敛速度")
print("2. 允许使用更大的学习率")
print("3. 避免数值计算问题")
print("4. 使不同尺度的特征具有可比性")

print("\n标准化 vs 归一化:")
print("- 标准化: 将特征转换为均值为0，标准差为1的分布")
print("- 归一化: 将特征缩放到[0, 1]或[-1, 1]的范围")
print("- 标准化更适合大多数机器学习算法")
print("- 归一化适合需要特征在特定范围的算法")